In [ ]:
%pip install -qU langchain langchain-openai langchain-community faiss-cpu tiktoken

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

import os

from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
if not os.getenv("OPENAI_API_KEY"):
    print("Error: OPENAI_API_KEY is not set in .env file")
    exit(1)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


# 把你的 OpenAI Key 放到环境变量（也可直接在系统环境里配置）
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# 可选：减少不必要的日志
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_PROJECT"] = "mq-retriever-demo"


In [4]:
import os
from pathlib import Path
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 你的txt路径（举例）
TXT_PATH = Path("data/kongyiji.txt")  # 改成你的真实路径

# 读取文本（处理编码问题）
text = TXT_PATH.read_text(encoding="utf-8", errors="ignore")

# 切分：chunk_size/overlap 可按需要调整
splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,    # 每块字符数
    chunk_overlap=40,  # 块间重叠，有助于跨段语义衔接
    separators=["\n\n", "\n", "。", "，", " ", ""],  # 中英混排时好用
)

chunks = splitter.split_text(text)
docs = [
    Document(page_content=chunk, metadata={"source": str(TXT_PATH), "chunk_id": i})
    for i, chunk in enumerate(chunks)
]

print(f"原文长度: {len(text)} 字符，切分得到 {len(docs)} 个文档块。示例：\n")
for d in docs[:2]:
    print(d.metadata, d.page_content[:120], "...\n")


原文长度: 2250 字符，切分得到 11 个文档块。示例：

{'source': 'data\\kongyiji.txt', 'chunk_id': 0} 鲁镇的酒店的格局，是和别处不同的：都是当街一个曲尺形的大柜台，柜里面预备着热水，可以随时温酒。做工的人，傍午傍晚散了工，每每花四文铜钱，买一碗酒，——这是二十多年前的事，现在每碗要涨到十文，——靠柜外站着，热热的喝了休息；倘肯多花一文，便可 ...

{'source': 'data\\kongyiji.txt', 'chunk_id': 1} 我从十二岁起，便在镇口的咸亨酒店里当伙计，掌柜说，我样子太傻，怕侍候不了长衫主顾，就在外面做点事罢。外面的短衣主顾，虽然容易说话，但唠唠叨叨缠夹不清的也很不少。他们往往要亲眼看着黄酒从坛子里舀出，看过壶子底里有水没有，又亲看将壶子放在热水里 ...



In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

emb = OpenAIEmbeddings(model="text-embedding-3-small")  # 或 text-embedding-3-large
vectorstore = FAISS.from_documents(documents=docs, embedding=emb)

# 保存到本地（可选）
INDEX_DIR = "faiss_index_from_txt"
vectorstore.save_local(INDEX_DIR)

retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
print("FAISS 索引构建完成（OpenAI embeddings）。")


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}